# OpenEO-SNAP InSAR workflow

This notebook contain an example on how to execute the OpenEO-SNAP InSAR workflow on a local machine with SNAP installed. This workflow can produce geocoded InSAR coherence and flattened + topographic corrected + multilooked + filtered interferogram

In [1]:
import urllib
import json
from IPython.display import JSON

from datetime import datetime

import glob
import os
import subprocess

## Input definition

In [2]:
burstId = 249435
sub_swath = 'IW2'
InSAR_pairs = [
    '20240809_20240821',
    '20240809_20240902',
    '20240821_20240902',
    '20240821_20240914'
]

# Coherence window size
cohWinRg, cohWinAz = (10, 2)

# Multillok parameters
nRgLooks, nAzLooks = (4, 1)

In [3]:
graph_filename = './graphs/interferogram_coh_2images_GeoTiff.xml'

In [4]:
if os.path.exists("CDSE_SECRET"):
    with open("CDSE_SECRET", "r") as file:
        lines = file.readlines()
    CDSE_ACCESS_KEY = lines[0].strip().split(": ")[1]
    CDSE_SECRET_KEY = lines[1].strip().split(": ")[1]
else:
    # get from environment vatiables:
    CDSE_ACCESS_KEY = os.environ.get("AWS_ACCESS_KEY_ID", None)
    CDSE_SECRET_KEY = os.environ.get("AWS_SECRET_ACCESS_KEY", None)


output_folder = '/home/mcallegari/PROJECT_DATA/ESA_ClouDInSAR/Test/s1_workflow/coh_interferogram'
if not os.path.exists(output_folder):
    output_folder = os.path.abspath("output")
print("output_folder: " + str(output_folder))

output_folder: /home/mcallegari/PROJECT_DATA/ESA_ClouDInSAR/Test/s1_workflow/coh_interferogram


## Burst download

In [5]:
date_list = [datetime.strptime(date, "%Y%m%d") for date in list(set('_'.join(InSAR_pairs).split('_')))]

for date in date_list:

    https_request = f"https://catalogue.dataspace.copernicus.eu/odata/v1/Bursts?$filter=" + urllib.parse.quote(
    f"ContentDate/Start ge {date.strftime("%Y-%m-%d")}T00:00:00.000Z and ContentDate/Start le {date.strftime("%Y-%m-%d")}T23:59:59.000Z and "
    f"PolarisationChannels eq 'VV' and "
    f"BurstId eq {burstId} and "
    f"SwathIdentifier eq '{sub_swath}'"
    )

    with urllib.request.urlopen(https_request) as response:
        content = response.read().decode()
    bursts = json.loads(content)
    ParentProductName = bursts['value'][0]['ParentProductName']

    burst_extract_cmd = (
        f"docker run -v {output_folder}:/home/ubuntu "
        f"-e AWS_ACCESS_KEY_ID={CDSE_ACCESS_KEY} -e AWS_SECRET_ACCESS_KEY={CDSE_SECRET_KEY} "
        f"cdse_utilities sentinel1_burst_extractor.sh -o /home/ubuntu "
        f"-n {ParentProductName} -p vv -s {sub_swath.lower()} -r {burstId}"
    )

    #print(burst_extract_cmd)
    os.system(burst_extract_cmd)
    

Input file size is 26039, 1510
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 26484, 1512
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 26484, 1512
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 26484, 1512
0...10...20...30...40...50...60...70...80...90...100 - done.


## Sentinel-1 processing

In [9]:
for pair in InSAR_pairs:

    mst_filename = glob.glob(os.path.join(output_folder, f'*{pair.split("_")[0]}*', 'manifest.safe'))[0]
    slv_filename = glob.glob(os.path.join(output_folder, f'*{pair.split("_")[1]}*', 'manifest.safe'))[0]
    
    mst_date, slv_date = [datetime.strptime(date, "%Y%m%d") for date in pair.split('_')]
    
    output_filename = os.path.join(output_folder, f'S1_phase_coh_{pair}.tif')
        
    phase_bandname = f'Phase_ifg_{sub_swath}_VV_{mst_date.strftime("%d%b%Y")}_{slv_date.strftime("%d%b%Y")}'
    coh_bandname = f'coh_{sub_swath}_VV_{mst_date.strftime("%d%b%Y")}_{slv_date.strftime("%d%b%Y")}'

    cmd = (
        f'gpt "{graph_filename}" -Pmst_filename="{mst_filename}" -Pslv_filename="{slv_filename}" '
        f'-PcohWinRg={cohWinRg} -PcohWinAz={cohWinAz} '
        f'-PnRgLooks={nRgLooks} -PnAzLooks={nAzLooks} '
        f'-Pphase_coh_bandnames="{phase_bandname},{coh_bandname}" '
        f'-Poutput_filename="{output_filename}"'
    )

    print(cmd)
    #os.system(cmd)

gpt "./graphs/interferogram_coh_2images_GeoTiff.xml" -Pmst_filename="/home/mcallegari/PROJECT_DATA/ESA_ClouDInSAR/Test/s1_workflow/coh_interferogram/S1A_SLC_20240809T170739_249435_IW2_VV_440377.SAFE/manifest.safe" -Pslv_filename="/home/mcallegari/PROJECT_DATA/ESA_ClouDInSAR/Test/s1_workflow/coh_interferogram/S1A_SLC_20240821T170739_249435_IW2_VV_442005.SAFE/manifest.safe" -PcohWinRg=10 -PcohWinAz=2 -PnRgLooks=4 -PnAzLooks=1 -Pphase_coh_bandnames="Phase_ifg_IW2_VV_09Aug2024_21Aug2024,coh_IW2_VV_09Aug2024_21Aug2024" -Poutput_filename="/home/mcallegari/PROJECT_DATA/ESA_ClouDInSAR/Test/s1_workflow/coh_interferogram/S1_phase_coh_20240809_20240821.tif"
gpt "./graphs/interferogram_coh_2images_GeoTiff.xml" -Pmst_filename="/home/mcallegari/PROJECT_DATA/ESA_ClouDInSAR/Test/s1_workflow/coh_interferogram/S1A_SLC_20240809T170739_249435_IW2_VV_440377.SAFE/manifest.safe" -Pslv_filename="/home/mcallegari/PROJECT_DATA/ESA_ClouDInSAR/Test/s1_workflow/coh_interferogram/S1A_SLC_20240902T170739_249435_IW2_